In [ ]:
import pandas as pd
df=pd.read_csv('/content/eng_3_train.tsv', delimiter='\t') 
df=df[["text", "category"]]
df = df.dropna()
df_dev=pd.read_csv('/content/eng_3_dev.tsv', delimiter='\t') 
df_dev=df_dev[["text", "category"]]
df_dev = df_dev.dropna()
df_test=pd.read_csv('/content/eng_3_test.tsv', delimiter='\t')
df_test = df_test.rename(columns={'text                        ': 'text'}) 
df_test = df_test.dropna()

In [ ]:
tedf=pd.read_csv('/content/tam_eng_train.tsv', delimiter='\t') 
tedf=tedf[["text", "category"]]
tedf = tedf.dropna()
tedf_dev=pd.read_csv('/content/tam_eng_dev.tsv', delimiter='\t') 
tedf_dev=tedf_dev[["text", "category"]]
tedf_dev = tedf_dev.dropna()
tedf_test=pd.read_csv('/content/tam_eng_test.tsv', delimiter='\t') 
tedf_test = tedf_test.dropna()


### Mixing english and tamil translated to english data

In [ ]:
train = df
valid = df_dev
test = tedf_test

In [ ]:
train = pd.concat([df, tedf])
valid = pd.concat([df_dev,tedf_dev])
test = pd.concat([tedf_test,df_test])

In [ ]:
print(len(train))
print(len(valid))
print(len(test))

5817
1456
1819


In [ ]:
print((train['category'].value_counts()))
print((valid['category'].value_counts()))
print((test['category'].value_counts()))

Non-anti-LGBT+ content    5017
Homophobic                 642
Transphobic                158
Name: category, dtype: int64
Non-anti-LGBT+ content    1257
Homophobic                 161
Transphobic                 38
Name: category, dtype: int64
Non-anti-LGBT+ content    1579
Homophobic                 194
Transphobic                 46
Name: category, dtype: int64


In [ ]:
concatenated = pd.concat([train,test,valid])

In [ ]:
concatenated['category'].value_counts()

Non-anti-LGBT+ content    7853
Homophobic                 997
Transphobic                242
Name: category, dtype: int64

In [ ]:
non_anti_lgbt_df = concatenated[concatenated['category'] == 'Non-anti-LGBT+ content']
homophobic_df = concatenated[concatenated['category'] == 'Homophobic']
transphobic_df = concatenated[concatenated['category'] == 'Transphobic']

In [ ]:
import pandas as pd
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.char as nac
import random

data = homophobic_df.copy()
syn_aug = naw.SynonymAug(aug_src='wordnet')
lis = []
for index, row in data.iterrows():
    text = syn_aug.augment(row['text'], n=2)
    label = [row['category']] * len(text)
    lis.extend(list(zip(text, label)))
tf = pd.DataFrame(lis, columns=['text', 'category'])
homophobic_df = pd.concat([data, tf], ignore_index=True)


In [ ]:
data = transphobic_df.copy()
syn_aug = naw.SynonymAug(aug_src='wordnet')
lis = []
for index, row in data.iterrows():
    text = syn_aug.augment(row['text'], n=5)
    label = [row['category']] * len(text)
    lis.extend(list(zip(text, label)))
tf = pd.DataFrame(lis, columns=['text', 'category'])
transphobic_df = pd.concat([data, tf], ignore_index=True)

In [ ]:
data = transphobic_df.copy()
aug = naw.RandomWordAug(action='insert')
lis = []
for index, row in data.iterrows():
    text = syn_aug.augment(row['text'], n=1)
    label = [row['category']] * len(text)
    lis.extend(list(zip(text, label)))
tf = pd.DataFrame(lis, columns=['text', 'category'])
transphobic_df = pd.concat([data, tf], ignore_index=True)

In [ ]:
print((transphobic_df['category'].value_counts()))
print((homophobic_df['category'].value_counts()))

Transphobic    2904
Name: category, dtype: int64
Homophobic    2991
Name: category, dtype: int64


In [ ]:
concatenated = pd.DataFrame(pd.concat([transphobic_df,homophobic_df,non_anti_lgbt_df.sample(2991)]))

In [ ]:
concatenated.category.value_counts()

Homophobic                2991
Non-anti-LGBT+ content    2991
Transphobic               2904
Name: category, dtype: int64